In [ ]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [ ]:
# Load environment variables in a file called .env
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")

# Check the key
if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

In [ ]:
openai = OpenAI()

In [ ]:
class FinvizWebsite():
    """
    Create this Website object from the given url using the BeautifulSoup library
    """
    
    def __init__(self, ticker):
        self.ticker = ticker.upper()
        self.url = f"https://finviz.com/quote.ashx?t={self.ticker}&p=d&ty=ea"
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
        }
        response = requests.get(self.url, headers=self.headers)
        soup = BeautifulSoup(response.content, "html.parser")
        self.title = soup.title.string if soup.title else "No title found"
        self.table = soup.find("table", class_="snapshot-table2")     

In [ ]:
def messages_for(website):
    system_prompt = """
        You are a financial analysis assistant that analyzes the contents of HTML formated table.
    and provides a summary of the stock's analysis with clear and professional language appropriate for financial research 
    with bulleted important list of **pros** and **cons** , ignoring text that might be navigation related. Repond in markdown.
    """
    
    user_prompt = f"""
        You are looking at a website titled {website.title}.\n
        The contents of this website is as follows; please provide a summary of the stock's analysis from this website in markdown.\n\n
        {website.table}
    """
    
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]


In [ ]:
def display_summary(ticker):
    website = FinvizWebsite(ticker)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    summary = response.choices[0].message.content
    display(Markdown(summary))

In [ ]:
display_summary("aapl")

In [ ]:
display_summary("tsla")